In [1]:
### baseline
# establish baseline: Epochs (Done)

### dataset
# construct bigger dataset
# add background images to reduce FP
# augment with other coral images (?)

### other changes
# perform preprocessing of images
# analyse and understand dataset
# different image sizes (Bigger for better detection)
# images with a number of different transformations/augmentations
# changing hyperparameters (https://github.com/ultralytics/yolov5/issues/607)
# different model sizes

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# creating a home variable

import os
import glob
import re
import seaborn as sns
import matplotlib.pyplot as plt

HOME = os.getcwd()
print(HOME)

/content


In [4]:
# installation of libraries

!pip install ultralytics==8.0.20
!pip install roboflow --quiet

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.5/107.7 GB disk)


In [5]:
from ultralytics import YOLO
from IPython.display import display, Image
from roboflow import Roboflow
import numpy as np

In [6]:
# load the dataset

!mkdir {HOME}/datasets
%cd {HOME}/datasets

rf = Roboflow(api_key="ask-ishan")
project = rf.workspace("ishan-nangia-v1cd3").project("coral-detection-ci")
dataset = project.version(2).download("yolov8")

/content/datasets
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.20, to fix: `pip install ultralytics==8.0.134`



Extracting Dataset Version Zip to Coral-Detection-CI-2 in yolov8:: 100%|██████████| 271/271 [00:00<00:00, 3941.65it/s]


In [ ]:
# train

%cd {HOME}
model = YOLO('yolov8m.pt')
results = model.train(data=f"{dataset.location}/data.yaml", epochs=300, plots=True)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

### Validation

In [ ]:
# if training hasn't been done

%cd {HOME}

from pathlib import Path
Path(f'{HOME}/runs/detect/train/weights/').mkdir( parents=True, exist_ok=True)

/content


In [7]:
## load model
# model = YOLO(f'{HOME}/runs/detect/train/weights/best.pt')

model = YOLO(f'/content/drive/MyDrive/Coastal Impact/yolo_model.pt')

In [ ]:
# validation
model.val(data=f"{dataset.location}/data.yaml")

### Metrics for our Results

In [8]:
def get_iou(box1, box2):
    """
    Implement the intersection over union (IoU) between box1 and box2

    Arguments:
        box1 -- first box, numpy array with coordinates (xmin, ymin, xmax, ymax)
        box2 -- second box, numpy array with coordinates
    """
    x11, y11, x21, y21 = box1
    x12, y12, x22, y22 = box2

    xi1 = max(x11, x12)
    yi1 = max(y11, y12)
    xi2 = min(x21, x22)
    yi2 = min(y21, y22)

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    # Calculate the Union area by using Formula: Union(A,B) = A + B - Inter(A,B)
    box1_area = (x21 - x11) * (y21 - y11)
    box2_area = (x22 - x12) * (y22 - y12)
    union_area = box1_area + box2_area - inter_area
    # compute the IoU
    iou = inter_area / union_area
    return iou

def generate_iou_plots(dict_iou, results_list):
    mean_ious = []
    len_ious = []
    std_ious = []
    for key, val in dict_iou.items():
      len_ious.append(len(val)/len(results_list))
      mean_ious.append(np.mean(val))
      std_ious.append(np.std(val))

    fig, ax = plt.subplots(2, 2, figsize=(15, 15), sharex=True)

    ax[0, 0].plot(iou_tp_ious.keys(), len_ious)
    ax[0,0].set_xlabel('IOU Thresholds')
    ax[0,0].set_ylabel('Number of IOU Recorded')
    ax[0,0].set_title(f"Number of IOU Recorded vs IOU thresholds on a per image basis")

    ax[0, 1].plot(iou_tp_ious.keys(), mean_ious)
    ax[0, 1].set_xlabel('IOU Thresholds')
    ax[0, 1].set_ylabel('Mean IOU')
    ax[0, 1].set_title(f"Mean IOU vs IOU thresholds on a per image basis")

    ax[1, 0].plot(iou_tp_ious.keys(), std_ious)
    ax[1, 0].set_xlabel('IOU Thresholds')
    ax[1, 0].set_ylabel('STD Dev. in IOU')
    ax[1, 0].set_title(f"Std Dev. in IOU vs IOU thresholds on a per image basis")

    scatter_xs = []
    scatter_ys = []
    for key, val in dict_iou.items():
      scatter_xs.extend([key]*len(val))
      scatter_ys.extend(val)

    ax[1, 1].scatter(scatter_xs, scatter_ys, alpha=0.5)
    ax[1, 1].plot(iou_tp_ious.keys(), mean_ious, color='red', linestyle='dotted')
    ax[1, 1].set_xlabel('IOU Thresholds')
    ax[1, 1].set_ylabel('Individual IOUs with line of mean')
    ax[1, 1].set_title(f"Individual IOUs vs IOU thresholds")

    return fig, ax

In [ ]:
# results
# results = model.predict(source=f"{dataset.location}/valid/images")

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


In [9]:
validation_image_names = list(glob.iglob("/content/datasets/Coral-Detection-CI-2/valid/images/*"))

In [10]:
# find a way around this for sure
### this goes through all the images one by one and gets their predictions right now. No batch inference.

results_list = []
for img in validation_image_names:
  result = model.predict(img)
  results_list.append(result[0])

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


In [124]:
import random

import cv2
from google.colab.patches import cv2_imshow

def draw_bboxes_xyxyn(bboxes, img):
  colors = [(150, 150, 150), (0,0,255)]
  drawn_img = img.copy()
  for i, box in enumerate(bboxes):
    x, y, x1, y1 = box
    x, x1 = x*img.shape[1], x1*img.shape[1]
    y, y1 = y*img.shape[0], y1*img.shape[0]
    cv2.rectangle(drawn_img, (int(x), int(y)), (int(x1), int(y1)), colors[i], 2)
  return drawn_img

In [125]:
iou_result_dict = {}
iou_result_per_image_dict = {}
iou_tp_ious = {}
iou_fp_condition_not_class_ious = {}
iou_fp_nonzero_ious = {}
iou_class_result_dict = {}
iou_class_result_per_image_dict = {}


%cd {HOME}
!rm -rf saved_images
!mkdir saved_images
image_counter = 0

iou_start = 0.5
iou_end = 1
for iou_thresh in np.arange(iou_start, iou_end, 0.1):
  tp_ious = []
  fp_condition_not_class_ious = []
  fp_nonzero_ious = []
  class_result_dict = {1:{'tp_count':0,
                          'fp_count':0,
                          'true_masks_with_match':0,
                          'true_masks_no_match':0,
                          'mask_count_mismatch':0,
                          "true_mask_count":0,
                          'pred_mask_count':0,
                          "true_multiple_matches":0,
                          "pred_multiple_matches":0},
                       0:{'tp_count':0,
                          'fp_count':0,
                          'true_masks_with_match':0,
                          'true_masks_no_match':0,
                          'mask_count_mismatch':0,
                          "true_mask_count":0,
                          'pred_mask_count':0,
                          "true_multiple_matches":0,
                          "pred_multiple_matches":0}
                       }
  result_dict = {'tp_count':0,
                'fp_count':0,
                'true_masks_with_match':0,
                'true_masks_no_match':0,
                'mask_count_mismatch':0,
                "true_mask_count":0,
                'pred_mask_count':0,
                "true_multiple_matches":0,
                "pred_multiple_matches":0}

  for idx, result_i in enumerate(results_list):
    img_name = validation_image_names[idx][:-3] + "txt"
    img_name = re.sub('images', 'labels', img_name)

    with open(img_name) as f:
      textfile = f.read()

    true_classes = [int(i[0]) for i in textfile.strip().split("\n")]
    true_masks = [[float(j) for j in i.split(' ')[1:]] for i in textfile.strip().split("\n")]

    predicted_classes = result_i.boxes.cls
    predicted_masks = result_i.boxes.xyxyn

    n_true_masks = len(true_masks)
    n_predicted_masks = result_i.boxes.xyxyn.shape[0]

    array_mask_match = np.zeros(shape=(n_predicted_masks, n_true_masks))
    coral_array_mask_match = np.zeros(shape=(list(predicted_classes).count(0),
                                             true_classes.count(0)))
    index_change_i = 0

    for idx_i, i_true_mask in enumerate(true_masks):
      index_change_j = 0

      x_i = i_true_mask[::2]
      y_i = i_true_mask[1::2]

      x_min = min(x_i)
      x_max = max(x_i)
      y_min = min(y_i)
      y_max = max(y_i)

      bbox_i = [x_min, y_min, x_max, y_max]

      current_true_class = true_classes[idx_i]
      if current_true_class==1:
        index_change_i = 1

      for idx_j, j_predicted_mask in enumerate(predicted_masks):
        j_predicted_mask = list(j_predicted_mask.numpy())
        iou = get_iou(bbox_i, j_predicted_mask)

        current_predicted_class = int(predicted_classes[idx_j].numpy())
        if current_predicted_class==1:
            index_change_j = 1

        condition_iou = iou>=iou_thresh
        condition_class = current_true_class==current_predicted_class

        if condition_iou and condition_class:
          # save image
          if np.random.randn()<0.01 and iou_thresh==iou_start:
            img_path = validation_image_names[idx]
            img = cv2.imread(img_path)
            bbox_image = draw_bboxes_xyxyn([bbox_i, j_predicted_mask], img)
            cv2.imwrite(f"saved_images/tp_{idx_i}_{idx_j}_idx_{idx}_iou_{iou}.png", bbox_image)


          array_mask_match[idx_j, idx_i] += 1
          if current_true_class==0:
            coral_array_idx_j = idx_j - index_change_j
            coral_array_idx_i = idx_i - index_change_i
            coral_array_mask_match[coral_array_idx_j, coral_array_idx_i] += 1
          result_dict["tp_count"] += 1
          class_result_dict[current_true_class]['tp_count'] += 1
          tp_ious.append(iou)
        elif condition_iou and (not condition_class):
          fp_condition_not_class_ious.append(iou)
        elif iou>0 and condition_class:
          fp_nonzero_ious.append(iou)


    true_results = array_mask_match.sum(axis = 0)
    pred_results = array_mask_match.sum(axis = 1)

    result_dict["fp_count"] += list(pred_results).count(0)

    result_dict["true_masks_with_match"] += true_results[true_results>=1].shape[0]
    result_dict["true_masks_no_match"] += list(true_results).count(0)
    result_dict["mask_count_mismatch"] += int(array_mask_match.shape[0]!=array_mask_match.shape[1])
    result_dict["true_mask_count"] += array_mask_match.shape[1]
    result_dict["pred_mask_count"] += array_mask_match.shape[0]
    result_dict["true_multiple_matches"] += true_results[true_results>1].shape[0]
    result_dict["pred_multiple_matches"] += pred_results[pred_results>1].shape[0]

    coral_true_results = coral_array_mask_match.sum(axis = 0)
    coral_pred_results = coral_array_mask_match.sum(axis = 1)

    class_result_dict[0]["fp_count"] += list(coral_pred_results).count(0)
    class_result_dict[0]["true_masks_with_match"] += coral_true_results[coral_true_results>=1].shape[0]
    class_result_dict[0]["true_masks_no_match"] += list(coral_true_results).count(0)
    class_result_dict[0]["mask_count_mismatch"] += int(coral_array_mask_match.shape[0]!=coral_array_mask_match.shape[1])
    class_result_dict[0]["true_mask_count"] += coral_array_mask_match.shape[1]
    class_result_dict[0]["pred_mask_count"] += coral_array_mask_match.shape[0]
    class_result_dict[0]["true_multiple_matches"] += coral_true_results[coral_true_results>1].shape[0]
    class_result_dict[0]["pred_multiple_matches"] += coral_pred_results[coral_pred_results>1].shape[0]

  iou_result_dict[iou_thresh] = result_dict
  iou_result_per_image_dict[iou_thresh] = {key: val/len(results_list) for key, val in result_dict.items()}

  iou_class_result_dict[iou_thresh] = class_result_dict
  # exclusively done for corals
  iou_class_result_per_image_dict[iou_thresh] = {key: val/len(results_list) for key, val in class_result_dict[0].items()}

  iou_tp_ious[iou_thresh] = tp_ious
  iou_fp_condition_not_class_ious[iou_thresh] = fp_condition_not_class_ious
  iou_fp_nonzero_ious[iou_thresh] = fp_nonzero_ious

/content


In [ ]:
fig, ax = generate_iou_plots(iou_fp_nonzero_ious, results_list)

In [ ]:
# to save the above plots

# %cd {HOME}
# fig.savefig('iou_fp_nonzero_ious_plots.png')

/content


What are the current problems with this model?
* With high IOU, low TPs: Corals TP go down with higher IOUs
* FP increases too for corals
* We are predicting more masks than the actual number of masks. Howevever, if we have at least some corals that haven't been labelled, then we our algorithm might be producing the correct amount of masks
* Most of our masks do get a match. At IOU 0.7, every two images we will get one mask that hasn't been matched properly.

In [ ]:
%cd {HOME}
print("DATA EXCL. FOR CORALS")
for key in iou_class_result_dict[0.5][0].keys():
  plt.plot(iou_class_result_per_image_dict.keys(), [val[key] for _, val in iou_class_result_per_image_dict.items()])
  plt.xlabel('IOU Thresholds')
  plt.ylabel(key)
  plt.title(f"{key} vs IOU thresholds on a per image basis")
  plt.savefig(f"plot_{key}.png")
  plt.show()

In [ ]:
%cd {HOME}

for key in iou_result_dict[0.5].keys():
  plt.plot(iou_result_per_image_dict.keys(), [val[key] for _, val in iou_result_per_image_dict.items()])
  plt.xlabel('IOU Thresholds')
  plt.ylabel(key)
  plt.title(f"{key} vs IOU thresholds on a per image basis")
  plt.savefig(f"plot_{key}.png")
  plt.show()

TO DO:

Checking validity of your metrics

Per class stats

common metrics: Recall/precision, etc.

How do your figures compare with whats already being computed by Yolo

Come up with all the cases/examples for these metrics and how they will change

### Testing

In [ ]:
# testing with selected model

## load model
model = YOLO(f'{HOME}/runs/detect/train/weights/best.pt')

# Perform object detection on an image using the model
## pure inference
results = model("/content/datasets/Coral-Detection-CI-2/valid/images/*", save=True, plots=True)

# Export the model to ONNX format
# success = model.export(format='onnx')

In [ ]:
# viewing detection results for specific images

import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.jpg')[10:15]:
      display(Image(filename=image_path, width=600))
      print("\n")

In [ ]:
%cd {HOME}
from ultralytics.utils.benchmarks import benchmark

# Benchmark on GPU
benchmark(model=f'{HOME}/runs/detect/train/weights/best.pt',
          data=f"{dataset.location}/data.yaml", imgsz=640, half=False, device=0)

### Rough

In [ ]:
%cd {HOME}

!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/valid/images save=True

In [ ]:
bboxes = [[311., 228., 392., 304.],
        [101., 372., 233., 466.],
        [594., 394., 640., 489.],
        [569.,  79., 640., 201.],
        [360., 383., 520., 489.],
        [161.,  67., 275., 181.]]